# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=gkey)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
cities = pd.read_csv("output_data/cities.csv")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,sao sebastiao,10,BR,1585186109,71,72.05,-23.76,-45.41,6.98
1,1,ushuaia,40,AR,1585185840,71,50.00,-54.80,-68.30,8.05
2,2,cidreira,0,BR,1585186110,78,72.27,-30.18,-50.21,15.55
3,3,butaritari,100,KI,1585186110,80,83.37,3.07,172.79,20.56
4,4,rundu,32,NaN,1585186110,100,68.00,-17.93,19.77,1.12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
# humidity = cities["Humidity"].astype(float)
# maxhumidity = humidity.max()
# locations = cities[["Lat", "Lng"]]


# Store latitude and longitude in locations
locations = cities[['Lat', 'Lng']]

# Fill NaN values and convert to float
humidity = cities["Humidity"]


In [47]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at a readable size

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [56]:
narrowed_city_df = cities.loc[(cities["Temp"] > 70) & (cities["Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
print(f"There are {len(narrowed_city_df)} cities satisfying the ideal weather condition.")
narrowed_city_df.head()




There are 6 cities satisfying the ideal weather condition.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,2,cidreira,0,BR,1585186110,78,72.27,-30.18,-50.21,15.55
1,329,xai-xai,0,MZ,1585186158,83,73.11,-25.05,33.64,9.82
2,408,morondava,0,MG,1585186169,85,78.30,-20.28,44.28,7.40
3,436,sao joao da barra,0,BR,1585186173,77,72.64,-21.64,-41.05,4.76
4,490,tomatlan,0,MX,1585186181,58,79.86,19.93,-105.25,7.02


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
hotels = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
narrowed_city_df["Hotel Name"] = hotels
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
0,2,cidreira,0,BR,1585186110,78,72.27,-30.18,-50.21,15.55,CIDREIRA
1,329,xai-xai,0,MZ,1585186158,83,73.11,-25.05,33.64,9.82,Xai-Xai District
2,408,morondava,0,MG,1585186169,85,78.30,-20.28,44.28,7.40,Morondava
3,436,sao joao da barra,0,BR,1585186173,77,72.64,-21.64,-41.05,4.76,Grussaí
4,490,tomatlan,0,MX,1585186181,58,79.86,19.93,-105.25,7.02,Tomatlán


In [58]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [59]:
 # Create hotel marker layer
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
markers = gmaps.marker_layer(locations)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…